Machine Learning Analytics on Reddit Database

Datacount: 3.5 million  
Datasize : 12GB

## Import Libraries

In [1]:
from pymongo import MongoClient

In [2]:
client = MongoClient('localhost', 27017)
db = client['reddit']
collection = db['posts_with_sentiment']

In [3]:
import pandas as pd
import json
from datetime import datetime

### Load Data

In [4]:
# cursor = collection.find()
# count = 0
# docs = []

# for doc in cursor:
#     comments = doc['comments']
    
#     if len(comments) >= 5:
        
#         post = {}
        
#         count = count + 1
        
#         post['pid'] = doc['id']
#         post['score'] = doc['score']
#         post['comments_count'] = doc['comments_count']
#         post['is_self'] = doc['is_self']

#         dt_obj = datetime.fromtimestamp(doc['timestamp'] / 1000)
#         post['hour'] = dt_obj.hour
#         post['subreddit'] = doc['subreddit']
        
#         comment_sent = 0
#         controv = 0
#         for comment in comments:
#             comment_sent = comment_sent + comment['sentiment']
#             controv = controv + comment['controversiality']
   
#         if comment_sent > 0:
#             sentiment = 'positive'
#         elif comment_sent < 0:
#             sentiment = 'negative'
#         else:
#             sentiment = 'neutral'
                
#         post['comments_sentiment'] = sentiment
#         post['controversiality'] = controv
        
#         docs.append(post)
        
#         if count % 10000 == 0:
#             print count
    
    
    
# df = pd.DataFrame.from_records(docs)
# len(df)

### Save data for faster access in future

In [25]:
#df.to_csv('data.csv')

### Load data from saved file

In [5]:
df = pd.read_csv('data.csv')

In [10]:
# df = df.drop('Unnamed: 0', 1)
df.head()

,comments_count,comments_sentiment,controversiality,hour,is_self,pid,score,subreddit,comments_sentiment1,is_self1,subreddit1,controversial,popular
0,247,positive,2,17,True,4g4dz7,1,pokemon,2,1,13301,1,0
1,13,positive,0,17,False,4g4dyf,1,hawks,2,0,11222,0,0
2,36,positive,0,17,True,4g4dys,1,AskReddit,2,1,621,0,0
3,7,negative,0,17,True,4g4dy1,1,NoMansSkyTheGame,0,1,5098,0,0
4,10,positive,0,17,True,4g4dxy,1,MtF,2,1,4813,0,0


### Among 3.5 million posts, there are 767818 Posts with comments count >= 5

In [11]:
len(df)

767818

### Categorize Strings into integers

In [12]:
array = df['comments_sentiment']
cats= pd.Categorical(array)
df['comments_sentiment1'] = cats.codes

array = df['is_self']
cats= pd.Categorical(array)
df['is_self1'] = cats.codes

array = df['subreddit']
cats= pd.Categorical(array)
df['subreddit1'] = cats.codes

In [13]:
df.head()

,comments_count,comments_sentiment,controversiality,hour,is_self,pid,score,subreddit,comments_sentiment1,is_self1,subreddit1,controversial,popular
0,247,positive,2,17,True,4g4dz7,1,pokemon,2,1,13301,1,0
1,13,positive,0,17,False,4g4dyf,1,hawks,2,0,11222,0,0
2,36,positive,0,17,True,4g4dys,1,AskReddit,2,1,621,0,0
3,7,negative,0,17,True,4g4dy1,1,NoMansSkyTheGame,0,1,5098,0,0
4,10,positive,0,17,True,4g4dxy,1,MtF,2,1,4813,0,0


In [14]:
def isControversial(x):
    if x > 0:
        return 1
    else:
        return 0
    
df['controversial'] = df['controversiality'].apply(isControversial)

### Define Functions
Popularity is defined by score > 36

In [15]:
def isPopular(x):
    if x > 36:
        return 1
    else:
        return 0

df['popular'] = df['score'].apply(isPopular)

In [16]:
df.head()

,comments_count,comments_sentiment,controversiality,hour,is_self,pid,score,subreddit,comments_sentiment1,is_self1,subreddit1,controversial,popular
0,247,positive,2,17,True,4g4dz7,1,pokemon,2,1,13301,1,0
1,13,positive,0,17,False,4g4dyf,1,hawks,2,0,11222,0,0
2,36,positive,0,17,True,4g4dys,1,AskReddit,2,1,621,0,0
3,7,negative,0,17,True,4g4dy1,1,NoMansSkyTheGame,0,1,5098,0,0
4,10,positive,0,17,True,4g4dxy,1,MtF,2,1,4813,0,0


### # of Popular posts

In [17]:
len(df[df['popular'] == 1])

235555

### Select necessary columns to apply prediction algorithm

In [18]:
df2 = df[['hour', 'subreddit1', 'comments_sentiment1', 'is_self1', 'controversial', 'popular']]

In [19]:
df2.head()

,hour,subreddit1,comments_sentiment1,is_self1,controversial,popular
0,17,13301,2,1,1,0
1,17,11222,2,0,0,0
2,17,621,2,1,0,0
3,17,5098,0,1,0,0
4,17,4813,2,1,0,0


In [20]:
data = df2.values[:]

In [21]:
data

array([[   17, 13301,     2,     1,     1,     0],
       [   17, 11222,     2,     0,     0,     0],
       [   17,   621,     2,     1,     0,     0],
       ..., 
       [    8, 13326,     2,     0,     1,     1],
       [    8, 14087,     2,     0,     0,     0],
       [    8,  8617,     2,     1,     0,     0]])

### Split into train set and test set

In [22]:
from sklearn.cross_validation import train_test_split

In [23]:
X_train, X_test, y_train, y_test = train_test_split(data[:, :5], data[:, 5], test_size=0.33, random_state=2)

In [24]:
X_train

array([[    5,  3383,     2,     1,     0],
       [   11,  2819,     2,     0,     0],
       [    2,  9067,     2,     1,     0],
       ..., 
       [   19,  4447,     2,     1,     0],
       [   22,  3164,     2,     0,     1],
       [   18, 13122,     0,     1,     0]])

In [25]:
y_train

array([0, 0, 1, ..., 1, 0, 0])

### Naive Bayes

In [26]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

clf_NB = GaussianNB()
nb_dec = clf_NB.fit(X_train, y_train)
output_NB = clf_NB.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_NB = accuracy_score(y_test, output_NB)
accuracy_NB

print classification_report(y_test,output_NB)

             precision    recall  f1-score   support

          0       0.85      0.79      0.82    175377
          1       0.59      0.68      0.63     78003

avg / total       0.77      0.76      0.76    253380



### SVM (takes too long, computer froze)

In [27]:
# from sklearn.svm import SVC
# clf_svm = SVC()
# svm_dec = clf_svm.fit(X_train, y_train)
# output_svm = clf_svm.predict(X_test)
# accuracy_svm = accuracy_score(y_test, output_svm)
# accuracy_svm

# print classification_report(y_test,output_svm)

### Random Forest Classifier (takes too long)

In [28]:
# from sklearn.ensemble import RandomForestClassifier
# clf_forest = RandomForestClassifier(n_estimators = 100, max_features = 5)
# rf_dec = clf_forest.fit(X_train, y_train)
# output_forest = clf_forest.predict(X_test)
# accuracy_forest = accuracy_score(y_test, output_forest)
# accuracy_forest

# print classification_report(y_test,output_forest)

### Logistic Regression

In [29]:
from sklearn.linear_model import LogisticRegression

clf_lr = LogisticRegression()
lr_score = clf_lr.fit(X_train, y_train)
output_lr = clf_lr.predict(X_test)
accuracy_lr = accuracy_score(y_test, output_lr)
accuracy_lr

print classification_report(y_test,output_lr)

             precision    recall  f1-score   support

          0       0.84      0.83      0.84    175377
          1       0.63      0.64      0.64     78003

avg / total       0.77      0.77      0.77    253380

